In [ ]:
%matplotlib inline
import os
import shutil
import pyemu
import helpers

In [ ]:
ies_d = "master_ies"
assert os.path.exists(ies_d)

In [ ]:
pst = pyemu.Pst(os.path.join(ies_d, "pest.pst"))

In [ ]:
oe = pst.ies.obsen0

In [ ]:
transforms = [{"type": "normal_score"}]

In [ ]:
dsi = pyemu.emulators.DSI(
    pst=pst,  # optional...
    data=oe,
    transforms=transforms,
    energy_threshold=0.9999,  # the truncated-svd energy threshold
    verbose=True,
)

In [ ]:
dsi.fit()

In [ ]:
dsi_d = "dsi_master"

In [ ]:
if os.path.exists(dsi_d):
    shutil.rmtree(dsi_d)

In [ ]:
dpst = dsi.prepare_pestpp(t_d=dsi_d, use_runstor=True)

In [ ]:
dpst

In [ ]:
dpst.pestpp_options["ies_num_reals"] = 300
dpst.control_data.noptmax = 3
dpst.pestpp_options.pop("parcov", None)
dpst.pestpp_options["ies_obs_en"] = pst.pestpp_options["ies_obs_en"]
shutil.copy2(
    os.path.join(ies_d, pst.pestpp_options["ies_obs_en"]),
    os.path.join(dsi_d, pst.pestpp_options["ies_obs_en"]),
)

dpst.pestpp_options["ies_phi_factor_file"] = pst.pestpp_options["ies_phi_factor_file"]
shutil.copy2(
    os.path.join(ies_d, pst.pestpp_options["ies_phi_factor_file"]),
    os.path.join(dsi_d, pst.pestpp_options["ies_phi_factor_file"]),
)
dpst.pestpp_options["ies_bad_phi_sigma"] = pst.pestpp_options["ies_bad_phi_sigma"]
dpst.pestpp_options

In [ ]:
dpst.write(os.path.join(dsi_d, "dsi.pst"))

In [ ]:
pyemu.os_utils.run("pestpp-ies dsi.pst /e", cwd=dsi_d)

In [ ]:
noptmax = 3

In [ ]:
helpers.plot_ies_timeseries(dsi_d, pst_name="dsi.pst", noptmax=noptmax)

In [ ]:
helpers.plot_ies_forecasts(dsi_d, pst_name="dsi.pst", include_t=True, noptmax=noptmax)

In [ ]:
tags = [
    "npf-k-layer1",
    "npf-k-layer5",
    "npf-k33-layer3",
    "sto-ss-layer1",
    "sto-sy-layer1",
]

In [ ]:
for tag in tags:
    helpers.plot_ies_properties(dsi_d, tag, pst_name="dsi.pst", noptmax=noptmax)